Cat Vs Dogs Kaggle dataset have some picture that are 2 channels or main format is BMP and they saved it as .jpg
in this notebook I try to fix dataset
in src folder I separate Dataset to train , val and test

In [1]:
import os
from pathlib import Path

from PIL import Image, UnidentifiedImageError
import cv2
import numpy as np

In [2]:
BASE_DIR = Path("/home/hamid/ML/Datasets/cat-vs-dog/split_data")
SUBSETS = ["train", "val", "test"]   # will iterate over train/val/test
CLASSES = ["Cat", "Dog"]             # folders inside each subset

In [9]:
#Helper Code : Use this to open one sample image and display it (for debugging / exploration):
def show_with_pil(path):
    img = Image.open(path)
    print(f"PIL: mode={img.mode}, size={img.size}, format={img.format}")
    img.show()   # opens an image viewer window

def show_with_cv2(path):
    img = cv2.imread(str(path), cv2.IMREAD_UNCHANGED)
    if img is None:
        print("OpenCV: cannot read image")
        return
    print(f"OpenCV: shape={img.shape}, dtype={img.dtype}")
    cv2.imshow("image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example: pick one image and inspect it
sample_path = BASE_DIR / "train" / "Dog" / os.listdir(BASE_DIR / "train" / "Dog")[103]
# show_with_pil(sample_path)
show_with_cv2(sample_path)

OpenCV: shape=(410, 500, 3), dtype=uint8


In [4]:
def is_valid_image(path: Path) -> bool:
    try:
        with Image.open(path) as img:
            img.verify()   # quick integrity check
        return True
    except (UnidentifiedImageError, OSError, SyntaxError):
        return False

In [5]:
def inspect_image(path: Path):
    info = {
        "path": path,
        "valid": True,
        "pil_format": None,
        "pil_mode": None,
        "channels": None,
        "bmp_data": False,
        "two_channels": False,
        "other_channels": False,
    }

    # 1) basic validity with PIL
    if not is_valid_image(path):
        info["valid"] = False
        return info

    # 2) open with PIL, read format/mode
    try:
        with Image.open(path) as img:
            info["pil_format"] = img.format  # e.g., 'JPEG', 'BMP', 'PNG'
            info["pil_mode"] = img.mode      # e.g., 'RGB', 'L', 'LA', 'RGBA'
    except Exception:
        info["valid"] = False
        return info

    if info["pil_format"] == "BMP":
        info["bmp_data"] = True

    # 3) open with OpenCV and check channels
    img_cv = cv2.imread(str(path), cv2.IMREAD_UNCHANGED)
    if img_cv is None:
        info["valid"] = False
        return info

    if img_cv.ndim == 2:
        channels = 1
    elif img_cv.ndim == 3:
        channels = img_cv.shape[2]
    else:
        channels = None

    info["channels"] = channels

    if channels == 2:
        info["two_channels"] = True
    elif channels is not None and channels != 3:
        info["other_channels"] = True

    return info


In [11]:
from pathlib import Path
import os
import cv2

BASE_DIR = Path("/home/hamid/ML/Datasets/cat-vs-dog/split_data")
SUBSETS = ["train", "val", "test"]
CLASSES = ["Cat", "Dog"]

def reencode_with_opencv(src_path: Path, quality: int = 95):
    img = cv2.imread(str(src_path), cv2.IMREAD_UNCHANGED)
    if img is None:
        print("Cannot read with OpenCV, skipping:", src_path)
        return
    ok = cv2.imwrite(str(src_path), img, [cv2.IMWRITE_JPEG_QUALITY, quality])
    if not ok:
        print("Cannot write with OpenCV, skipping:", src_path)

for subset in SUBSETS:
    for cls in CLASSES:
        folder = BASE_DIR / subset / cls
        for fname in os.listdir(folder):
            path = folder / fname
            if not path.is_file():
                continue
            # only target JPEG-like files
            if path.suffix.lower() not in [".jpg", ".jpeg", ".jpe"]:
                continue
            print("Re-encoding:", path)
            reencode_with_opencv(path, quality=95)


Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/1735.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/8411.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/6473.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/6093.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/9279.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/5781.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/3671.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/11654.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/863.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/8681.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/2698.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/7436.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train

Corrupt JPEG data: 104 extraneous bytes before marker 0xd9


Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/4353.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/750.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/8275.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/272.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/8314.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/672.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/8053.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/9342.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/11982.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/881.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/3691.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/8028.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Ca

Corrupt JPEG data: 1158 extraneous bytes before marker 0xd9


Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/346.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/7675.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/5619.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/762.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/4134.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/12133.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/10428.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/7749.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/373.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/6795.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/10678.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Cat/12432.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/trai

...

Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/11715.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/11761.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/2595.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/10386.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/2087.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/4950.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/6039.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/11848.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/11548.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/9995.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/2816.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/10259.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data

Corrupt JPEG data: 402 extraneous bytes before marker 0xd9


Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/2918.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/8992.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/2631.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/9980.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/6656.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/6155.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/8256.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/5299.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/11011.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/10930.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/7673.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/train/Dog/12037.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/tr

Corrupt JPEG data: 164 extraneous bytes before marker 0xd9


Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/val/Dog/5641.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/val/Dog/7848.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/val/Dog/11441.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/val/Dog/597.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/val/Dog/3157.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/val/Dog/10157.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/val/Dog/1530.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/val/Dog/2704.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/val/Dog/5214.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/val/Dog/379.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/val/Dog/5091.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/val/Dog/1298.jpg
Re-encoding: /home/hamid/ML/Datasets/cat-vs-dog/split_data/val/Dog/12076.jpg
Re-encoding

In [12]:
#collect bad images
bad_invalid = []
bad_bmp = []
bad_two_channels = []
bad_other_channels = []

for subset in SUBSETS:
    for cls in CLASSES:
        folder = BASE_DIR / subset / cls
        for fname in os.listdir(folder):
            path = folder / fname

            # skip non-files (just in case)
            if not path.is_file():
                continue

            info = inspect_image(path)

            if not info["valid"]:
                bad_invalid.append(info["path"])
                continue

            if info["bmp_data"]:
                bad_bmp.append(info["path"])

            if info["two_channels"]:
                bad_two_channels.append(info["path"])

            elif info["other_channels"]:
                bad_other_channels.append(info["path"])

print("Invalid images (cannot be opened):")
for p in bad_invalid:
    print("  ", p)

print("\nImages with BMP data:")
for p in bad_bmp:
    print("  ", p)

print("\nImages with 2 channels:")
for p in bad_two_channels:
    print("  ", p)

print("\nImages with channels != 3 (excluding 2‑channel already listed):")
for p in bad_other_channels:
    print("  ", p)


Invalid images (cannot be opened):

Images with BMP data:

Images with 2 channels:

Images with channels != 3 (excluding 2‑channel already listed):


In [13]:
OUTPUT_ROOT = BASE_DIR / "clean_rgb"   # new cleaned copy
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

def convert_to_rgb_jpeg(src_path: Path, dst_path: Path):
    dst_path.parent.mkdir(parents=True, exist_ok=True)
    with Image.open(src_path) as img:
        img = img.convert("RGB")
        img.save(dst_path, format="JPEG", quality=95)

for subset in SUBSETS:
    for cls in CLASSES:
        in_folder = BASE_DIR / subset / cls
        out_folder = OUTPUT_ROOT / subset / cls

        for fname in os.listdir(in_folder):
            src = in_folder / fname
            if not src.is_file():
                continue

            rel = src.relative_to(BASE_DIR)
            dst = OUTPUT_ROOT / rel

            # Inspect to see if it is bad
            info = inspect_image(src)
            if (not info["valid"]) or info["bmp_data"] or info["two_channels"] or info["other_channels"]:
                print("Fixing:", src)
                try:
                    convert_to_rgb_jpeg(src, dst.with_suffix(".jpg"))
                except Exception as e:
                    print("  Failed to fix:", src, "error:", e)
            else:
                # Copy “good” images as they are (or convert all to JPEG if you prefer)
                convert_to_rgb_jpeg(src, dst.with_suffix(".jpg"))
